In [8]:
import jax.numpy as jnp
import numpy as np
def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)

In [9]:
x=np.array([0,0,0,0,1,1,1])
k=2

In [10]:
one_hot(x,k)

DeviceArray([[1., 0.],
             [1., 0.],
             [1., 0.],
             [1., 0.],
             [0., 1.],
             [0., 1.],
             [0., 1.]], dtype=float32)

In [11]:
import numpy as np
from pysat.formula import CNF
def do_it(instance_name):
        #cnf = CNF(from_file=(instance_name + ".cnf"))
        #print(cnf.clauses[0][0])

        target_name = instance_name + "_samples_sol.npy"
        target_func=np.load(target_name)


        #number_of_unsat=check_clauses(cnf.clauses,target_func)
        #print(number_of_unsat)
        #, target_func),axis=0)
        #print(violated_constraints_for_candidates)
        weights= 1#tbf    
        return((target_func,weights))

def assignment(x):
    if x > 0:
        return False
    elif x < 0:
        return True
    else:
        print("error, not a proper input")

def current_assignment(candidate, index):
        return candidate[index]

def check_clauses(clauses,candidate):
        count_unsat=0
        print(clauses[0][0])
        for i in range(len(clauses)):
                satisfied=0
                for j in range(len(clauses[i])):
                        if satisfied==0:
                                desired=assignment(clauses[i][j])
                                print("a",desired)
                                current= current_assignment(candidate[i],j)
                                print("b",current)
                                if desired==current:
                                        satisfied=1
                        if satisfied==0:
                                count_unsat+= 1
        return count_unsat

In [12]:
instance_name="/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/git_multiple_candidates/ml_based_sat_solver/Data/one_example/41-20486"

a=do_it(instance_name)

In [13]:
print(a)

(array([[False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [False, False,  True, ..., False,  True, False],
       ...,
       [False, False,  True, ...,  True,  True,  True],
       [False,  True, False, ...,  True, False,  True],
       [False,  True,  True, ...,  True,  True,  True]]), 1)


In [1]:
import glob
import gzip
from os.path import join, exists, splitext
import pickle

import jraph
import nnf
from func_timeout import func_timeout, FunctionTimedOut
from torch.utils import data
import numpy as np
from pysat.formula import CNF
from collections import namedtuple
from constraint_problems import get_problem_from_cnf
from random_walk import violated_constraints
from jax import vmap


def create_candidates_with_sol(data_dir, sample_size, threshold):
    solved_instances = glob.glob(join(data_dir, "*_sol.pkl"))
    for g in solved_instances:
        with open(g, "rb") as f:
            p = pickle.load(f)
        n = np.array(list(p.values()), dtype=bool)
        samples = sample_candidates(n, sample_size-1, threshold)
        #print(samples)
        samples = np.concatenate((np.reshape(n,(1,len(n))),samples),axis=0)
        name = g.split('_sol.pkl')[0]
        with open(name + "_samples_sol.npy", "wb") as f:
            np.save(f, samples)
    #return(samples)



def sample_candidates(original, sample_size, threshold):
    np.random.seed(sum(original))
    condition = np.random.random((sample_size, original.shape[0])) < threshold
    return np.where(condition, np.invert(original), original)



In [3]:


data_dir="/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset"
sample_size=2
threshold=0.01

create_candidates_with_sol(data_dir, sample_size, threshold)

In [16]:
import time
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from torch.utils import data

from data_utils import SATTrainingDataset, JraphDataLoader
from model import network_definition    
    
path='/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset/'
sat= SATTrainingDataset(path)
print("loaded data sucessfully")




loaded data sucessfully


In [ ]:
problem=sat[0][0]
candidates=sat[0][1][0:2]
#weights=sat[2][2]
#print(weights)
#print(sat[0][0])
print(candidates)


(array([[False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False,  True, False],
       [ True,  True,  True, ...,  True,  True, False],
       ...,
       [False,  True, False, ...,  True, False, False],
       [False,  True,  True, ...,  True,  True, False],
       [ True, False, False, ...,  True, False,  True]]), DeviceArray([    3, 75409, 76401, 72834, 75006, 76637, 76217, 76549,
             76158, 77314, 74121, 74437, 77201, 76549, 74921, 75992,
             76554, 77580, 75082, 77341], dtype=int32))


In [18]:
print(candidates[1])

[    3 75409 76401 72834 75006 76637 76217 76549 76158 77314 74121 74437
 77201 76549 74921 75992 76554 77580 75082 77341]


In [2]:
import time
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from torch.utils import data

from data_utils import SATTrainingDataset, JraphDataLoader
from model import network_definition

NUM_EPOCHS = 10
f=0.1

# # Make a batched version of the forwarding
# batched_predict = jax.vmap(network.apply, in_axes=(None, 0))


# def loss(params, problems, targets):
#     preds = batched_predict(params, problems)
#     return -jnp.mean(preds * targets)


def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)

vmap_one_hot=jax.vmap(one_hot, in_axes=(0,None), out_axes=0)

path='/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset/'
sat_data = SATTrainingDataset(path)
train_data, test_data = data.random_split(sat_data, [0.8, 0.2])

train_loader = JraphDataLoader(train_data, batch_size=1, shuffle=True)

network = hk.without_apply_rng(hk.transform(network_definition))
params = network.init(jax.random.PRNGKey(42), sat_data[0][0].graph)

opt_init, opt_update = optax.adam(1e-3)
opt_state = opt_init(params)



2022-12-22 14:11:51.281777: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2022-12-22 14:11:52.162901: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 1.885859s
Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necess

In [4]:
import numpy as np

vmap_one_hot=jax.vmap(one_hot, in_axes=(0,None), out_axes=0)

@jax.jit
def compute_log_probs(decoded_nodes, mask, candidate):
        #return jax.nn.log_softmax(decoded_nodes) * candidate ###this is incompatible regarding the dimensions
        #a=jax.nn.log_softmax(decoded_nodes) * one_hot(candidate,2)
        #print("c",np.shape(candidate))
        #print("1hc",np.shape(one_hot(candidate,2)))
        print(np.shape(decoded_nodes))
        print(np.shape(mask[:, None]))
        a=jax.nn.log_softmax(decoded_nodes) * mask[:, None] #does not even return, wrong dimensions while calculating but (2m, 2), (m, 1)
        #a= jnp.tensordot(jax.nn.log_softmax(decoded_nodes), candidate) ###gives wrong dimension but returns
        #b=jnp.where(jax.nn.log_softmax(decoded_nodes),one_hot(mask[:, None],2),size=len(candidates[0]))
        print(np.shape(a))
        print(np.shape(candidate))
        b= jnp.dot(candidate,a.T)
        #a=jnp.tensordot(candidate, jax.nn.log_softmax(decoded_nodes) * mask[:, None],2) 
        print(np.shape(b))
        #a=1
        #log_probs=jnp.tensordot(candidate, a)
        #print(np.shape(log_probs))
        return b

vmap_compute_log_probs=jax.vmap(compute_log_probs, in_axes=(None,None, 0), out_axes=0)



@jax.jit
def update(params, opt_state, x, y, f):
        #g = jax.grad(prediction_loss)(params, *x, y)

        #TBD!!! -> change this function here

        #g=jax.grad(new_prediction_loss)(params, *x, *y, f)
        #print(y)
        #c = y[0]
        #e = y[1]
        
        #c,e = zip(*y)
        #print(c[0])
        #print(e[0])
        #print(x)
        batch_masks=x[0]
        batch_graphs=x[1]
        batch_c=y[0]
        batch_e=y[1]
        #print(len(batch_masks))
        #print(len(batch_graphs))
        #print(len(batch_c))
        #print(len(batch_e))
        print(batch_e[0])
        print(batch_e)
        g=jax.grad(new_prediction_loss)(params, batch_masks[0], batch_graphs[0], batch_c[0], batch_e[0], f)
        #g=jax.grad(batched_loss)(params, *x, c, e, f)
        ####

        updates, opt_state = opt_update(g, opt_state)
        return optax.apply_updates(params, updates), opt_state


@jax.jit
def prediction_loss(params, mask, graph, solution):
        decoded_nodes = network.apply(params, graph)
        solution = one_hot(solution, 2)
        # We interpret the decoded nodes as a pair of logits for each node.
        log_prob = jax.nn.log_softmax(decoded_nodes) * solution
        return -jnp.sum(log_prob * mask[:, None]) / jnp.sum(mask)


@jax.jit
def new_prediction_loss_single(params, mask, graph, candidate, energy, f: float):
        decoded_nodes = network.apply(params, graph)
        candidate = one_hot(candidate, 2)
        # We interpret the decoded nodes as a pair of logits for each node.
        log_prob = jax.nn.log_softmax(decoded_nodes) * candidate
        weights = jax.nn.softmax(- f * energy)
        weighted_log_probs = jnp.dot(log_prob, weights)
        return -jnp.sum(weighted_log_probs * mask[:, None]) / jnp.sum(mask)

@jax.jit
def new_prediction_loss(params, mask, graph, candidates, energies, f: float):
        decoded_nodes = network.apply(params, graph)
        print("candidates",np.shape(candidates))
        #print(np.shape(candidates))
        candidates = vmap_one_hot(candidates, 2)
        
        #print(np.shape(candidates))
        # We interpret the decoded nodes as a pair of logits for each node.
        #print(np.shape(jax.nn.log_softmax(decoded_nodes)))
        #print(np.shape(candidates[0]))
        #print(np.shape(mask[:,None]))
        #print(len(candidates))
        #print("shape",np.shape(jnp.tensordot(candidates[0],jax.nn.log_softmax(decoded_nodes) * mask[:, None])))
        #print("shape", np.shape(jax.nn.log_softmax(decoded_nodes) * one_hot(mask[:, None],2)))
        #print("mask",np.shape(mask[:,None]))
        #print("mask[:,None]",np.shape(mask))
        #print("decoded_nodes",np.shape(decoded_nodes))
        #print("one_hot_candidates",np.shape(candidates))
        #print()
        #c=np.tensordot(jax.nn.log_softmax(decoded_nodes)*mask[:, None],2)
        #c=jax.nn.log_softmax(decoded_nodes)*mask[:, None]
        #print("c",np.shape(c))
        #log_prob=jax.vmap(np.tensordot,in_axes=(0,None),out_axes=0)(candidates, c)
        log_prob=vmap_compute_log_probs(decoded_nodes, mask, candidates)
        #print("log_prob",np.shape(log_prob))
        #print(np.shape(candidates))
        
        ####
        #og_prob = jax.nn.log_softmax(decoded_nodes) * candidates
        #print(np.shape(log_prob))

        #####
        #print(energies)
        #print(np.shape(energies))
        weights = jax.nn.softmax(- f * energies)
        #print("w",np.shape(weights))

        weighted_log_probs = jax.vmap(jnp.dot,axis_name=(0,0), out_axes=0)(log_prob, weights)
        #print(np.shape(weighted_log_probs))
        summed_weighted_log_probs=np.sum(weighted_log_probs, axis=0) #sum over all candidates
        #print(np.shape(summed_weighted_log_probs))
        loss=-jnp.sum(summed_weighted_log_probs @ mask[:, None]) / jnp.sum(mask)
        #print(np.shape(loss))
        #print(loss)
        return loss



        

batched_loss = jax.vmap(new_prediction_loss, in_axes=(0, 0, 0, 0, 0, None), out_axes=0)


In [5]:
for (batch_p, batch_ce) in train_loader:
    #batch_masks=batch_p[0]
    #batch_graphs=batch_p[1]
    #batch_c=batch_ce[0]
    batch_e=batch_ce[1]
    #print(len(batch_masks))
    #print(len(batch_graphs))
    #print(len(batch_c))
    print(len(batch_e))
    #print(batch_e)
    params, opt_state = update(params, opt_state, batch_p, batch_ce, f)

2022-12-20 11:42:34.101018: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 2s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2022-12-20 11:42:34.481382: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 2.382834s
Constant folding an instruction is taking > 2s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necess

1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [ ]:
for (p, ce) in train_loader:
    for i in range(0,len(ce[1])):
            params, opt_state = update(params, opt_state, p, ce[:][i], f)
            


candidates (20, 13262)
mask (822936, 1)
mask[:,None] (822936,)
decoded_nodes (1645872, 2)
one_hot_candidates (20, 13262, 2)


NameError: name 'c' is not defined

In [3]:
path='/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset/'
sat_data = SATTrainingDataset(path)
from jax.numpy import asarray

In [4]:
problem, ce = sat_data[0]

In [25]:
print(problem.mask.shape)
print(problem.params)
print(problem.graph.nodes.shape)

(411468,)
(18541, 323640, 23)
(411468, 2)


In [5]:
print(ce[0].shape)

(2, 18541)


In [26]:
train_loader = JraphDataLoader(sat_data, batch_size=1, shuffle=True)

In [7]:
for i, (p, ce) in enumerate(train_loader):
    if i==0:
        print(p[0].shape)
        print(p[1].nodes.shape)
    
    #for i in range(0,len(ce[1])):
    #        params, opt_state = update(params, opt_state, p, ce[:][i], f)

(205734,)
(411468, 2)


KeyboardInterrupt: 

In [27]:
def collate_fn(batch):
    problems, tuples = zip(*batch)
    candidates, energies = zip(*tuples) ###this might be the problem...
    masks, graphs = zip(*((np.fromstring(p.mask), p.graph) for p in problems))
    return (np.concatenate(masks), jraph.batch(graphs)), (candidates, asarray(energies))#(list(zip(*candidates)), np.concatenate(jax.numpy.asarray(energies)))

In [30]:
p,ce=sat_data[1]


In [51]:

for (p, ce) in train_loader:
    #print(ce)
    c=ce[0]
    e=ce[1]
    print(len(e))
    for i in range(0,2):#len(ce)):
        #print(c[i])
        #print(e[i])
        print(ce[:][i])

4
(array([[False, False, False, ..., False, False, False],
       [ True, False,  True, ...,  True,  True, False],
       [False,  True, False, ...,  True, False, False],
       ...,
       [False,  True,  True, ...,  True, False, False],
       [False, False,  True, ...,  True,  True, False],
       [ True,  True,  True, ...,  True,  True,  True]]), array([[False, False, False, ..., False, False, False],
       [False,  True,  True, ...,  True, False, False],
       [ True,  True,  True, ...,  True, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [ True, False, False, ...,  True, False, False],
       [ True, False, False, ..., False,  True, False]]), array([[False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False,  True],
       [False,  True,  True, ...,  True, False,  True],
       ...,
       [False,  True, False, ..., False,  True, False],
       [ True, False, False, ...,  True,  True,  True],
     

KeyboardInterrupt: 

In [ ]:

print("Entering training loop")

for epoch in range(NUM_EPOCHS):
        start_time = time.time()
        for (x, y) in train_loader:
            params, opt_state = update(params, opt_state, x, y)
            # print(f"{batch} done")
        epoch_time = time.time() - start_time

        # train_acc = accuracy(params, train_images, train_labels)
        # test_acc = accuracy(params, test_images, test_labels)
        print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))

        #test_acc = jnp.mean(jnp.asarray([prediction_loss(params, p.mask, p.graph, s) for (p, s) in test_data]))

        #TBD!!!

        test_acc = jnp.mean(jnp.asarray([new_prediction_loss(p, p.graph, c, f) for (p, c) in test_data]))
        
        ##

        # print("Training set accuracy {}".format(train_acc))
        print("Test set accuracy {}".format(test_acc))

    # TODO: Save the model here


print("done")